In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import logging
import gzip
import gensim 
import re
import spacy
import math
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [34]:
print(torch.cuda.is_available())
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

True


'cuda:0'

# Preprocessing & Loading data

## Preprocess 

### Clothing Review Dataset

In [35]:
# Load dataset for clothing reviews
reviews = pd.read_csv("/content/drive/MyDrive/data /Womens Clothing E-Commerce Reviews.csv")
reviews = reviews.dropna()
print(reviews.shape)
reviews.head()

(19662, 11)


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits


In [36]:
reviews['Title'] = reviews['Title'].fillna('')
reviews['Review Text'] = reviews['Review Text'].fillna('')
reviews['review'] = reviews['Title'] + ' ' + reviews['Review Text']

In [37]:
reviews = reviews[['review', 'Rating']]
reviews.columns = ['review', 'rating']
reviews.head()

,review,rating
2,Some major design flaws I had such high hopes ...,3
3,"My favorite buy! I love, love, love this jumps...",5
4,Flattering shirt This shirt is very flattering...,5
5,Not for the very petite I love tracy reese dre...,2
6,Cagrcoal shimmer fun I aded this in my basket ...,5


#### Tokenize each sentence

In [38]:
#take advantage of nltk to tokenize all sentences
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
en_stop_words = set(stopwords.words('english'))
tokenizer = nltk.RegexpTokenizer(r'\w+|\$[\d\.]+')

def tokenize_sent(sent):
    
    tokenized = tokenizer.tokenize(sent)
    filtered = [w.lower() for w in tokenized if w.lower() not in en_stop_words]
    return filtered

reviews['tokenized'] = reviews['review'].apply(lambda x: tokenize_sent(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
# Sanity cehck for tokenizers
print(reviews.head(3))

                                              review  ...                                          tokenized
2  Some major design flaws I had such high hopes ...  ...  [major, design, flaws, high, hopes, dress, rea...
3  My favorite buy! I love, love, love this jumps...  ...  [favorite, buy, love, love, love, jumpsuit, fu...
4  Flattering shirt This shirt is very flattering...  ...  [flattering, shirt, shirt, flattering, due, ad...

[3 rows x 3 columns]


#### Encode and truncate sentence

In [40]:
# get unique words in the corpus
all_words = []
for x in reviews['tokenized']:
    all_words.extend(x)

word_set = list(set(all_words))
word_count = Counter(all_words)

# filter out words with low frequency
for word_list in reviews.tokenized:
    new_list = []
    for word in word_list:
        if word_count[word] > 2:
            new_list.append(word)
    word_list = new_list

# update set of words after removing the ones with low frequency
new_word_list = []
for x in reviews['tokenized']:
    new_word_list.extend(x)
word_set = list(set(new_word_list))

# map each unique words & unknown token in reviews.encoded to an index
word2index = {}
word2index['<UNK>'] = 0
word2index['<PAD>'] = 1

for i, word in enumerate(word_set, 2):
    word2index[word] = i

# encode the original sequence
def encode(sent_list):
    result = []
    for x in sent_list:
        index = word2index[x]
        result.append(index)
    return result

reviews['encoded'] = reviews.tokenized.apply(lambda x: encode(x))

print(reviews.head())

# get sequence average length
total_len = 0
for x in reviews.encoded:
    total_len += len(x)

ave_len = math.floor(total_len/reviews.shape[0])

# filter out long sequences --> encode all sequence to length = ave_len
# pad short sequence

for i, row in reviews.iterrows():
    size = min(len(row.encoded), ave_len)
    new_encoded = row.encoded[:size]
    if size < ave_len:
        for j in range(0, ave_len - len(row.encoded)):
            new_encoded.append(1)
    reviews.at[i, 'encoded'] = new_encoded

reviews['review_length'] = reviews.encoded.apply(lambda x: len(x))

                                              review  ...                                            encoded
2  Some major design flaws I had such high hopes ...  ...  [2878, 13858, 13950, 12209, 10213, 3101, 135, ...
3  My favorite buy! I love, love, love this jumps...  ...  [6918, 6004, 7255, 7255, 7255, 12126, 11136, 4...
4  Flattering shirt This shirt is very flattering...  ...  [2724, 1758, 1758, 2724, 12396, 7933, 10849, 1...
5  Not for the very petite I love tracy reese dre...  ...  [13875, 7255, 7401, 2828, 1761, 12799, 13875, ...
6  Cagrcoal shimmer fun I aded this in my basket ...  ...  [152, 13756, 11136, 7457, 9013, 2222, 10273, 9...

[5 rows x 4 columns]


In [41]:
# sanity check for encoding:
print(len(all_words))
print(ave_len)
lengths = [len(x) for x in reviews.encoded]
print(max(lengths))
print(min(lengths))
print(set(reviews.rating))
review_length = reviews.review_length
reviews.head()
print(len(word_set))

639434
32
32
32
{1, 2, 3, 4, 5}
14088


In [42]:
# Train test split from skearln
data_size = len(reviews['encoded'])
assert data_size == len(reviews['rating']) 
X, y = list(zip(list(reviews['encoded']),(list(reviews['review_length'])))), list(reviews['rating'])
# X(data, length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [43]:
# Sanity check for length match
print(X_train[0])

([5577, 10039, 11559, 5558, 8449, 10039, 4244, 11223, 6004, 307, 5577, 5558, 8892, 10042, 11530, 7255, 11358, 8800, 7315, 453, 3117, 307, 28, 4982, 5558, 8892, 7086, 2484, 12712, 28, 6400, 8892], 32)


## Dataset and Dataloader

### Clothing Review Dataset

In [44]:
class Dataset_Glove(Dataset):
    def __init__(self, X, y): 
        self.X = X
        y_new = [i-1 for i in y]
        self.y = y_new

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx]

In [45]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [46]:
print(trainSet_glv[0])

(([5577, 10039, 11559, 5558, 8449, 10039, 4244, 11223, 6004, 307, 5577, 5558, 8892, 10042, 11530, 7255, 11358, 8800, 7315, 453, 3117, 307, 28, 4982, 5558, 8892, 7086, 2484, 12712, 28, 6400, 8892], 32), 4)


In [47]:
def myCollate(batch):
    data = [item[0] for item in batch]
    label = [item[1] for item in batch]
    # sort sequence according to it's length in a batch
    data = data
    # data.sort(key=lambda x: x[1], reverse = True)

    # data (review: list, length: int)
    review = torch.tensor([x[0] for x in data], dtype = torch.long)
    label = torch.tensor(label, dtype = torch.long)
    seq_len = [x[1] for x in data]
    return review, label, seq_len

In [48]:
trainSet_glv = Dataset_Glove(X_train, y_train)
testSet_glv = Dataset_Glove(X_test, y_test)

In [49]:
trainLoader_glv = DataLoader(dataset = trainSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
testLoader_glv = DataLoader(dataset = testSet_glv, batch_size = 16, collate_fn = myCollate, shuffle = True, drop_last = True)
it = iter(trainLoader_glv)
x, y, seq_len = it.next()

In [50]:
x.data

tensor([[ 8892,  1774,  3817,  5418,  1774,  6341,  1758,  9731,  1749, 11573,
         13399,  3322,  1758,  5558,  9592,  1336,  2934,  1344,  1113,  5646,
          3173, 12738, 11427,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [ 1245, 12712, 12741,  1245,   395, 12741,   135, 12712, 10474, 13316,
         11345,  2998,   441, 11345,  4046, 10507, 10238,  3751, 12715, 12762,
          4359,  8587, 12762,  7255, 10240,  7942, 10166,  4400,  1574,  2255,
          2169,  2420],
        [13879,  1245, 10432,  4129,  3708, 11003, 12712,   395, 10432,  2907,
          1208,  2668,  8138,  3453,  3016,  8589, 12209,  9875, 13557,  7878,
          3380,  3117,  7878, 13557,     1,     1,     1,     1,     1,     1,
             1,     1],
        [ 9714,  9714,  7727,  1749, 13858,  1113, 12585, 12920,  9229,  5618,
          4638,  9798,  2147, 13249,  8866,  8892,   559, 12315,     1,     1,
             1,     1,     1,     1,     1,     1,     1,  

In [51]:
y

tensor([0, 4, 3, 1, 4, 3, 4, 4, 3, 1, 4, 2, 3, 4, 3, 3])

In [52]:
seq_len

[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]

## Embedding

In [53]:
def load_glove_vectors(glove_file="/content/drive/MyDrive/data /glove.6B.50d.txt"):
    """Load the glove word vectors"""
    word_vectors = {}
    with open(glove_file) as f:
        for line in f:
            split = line.split()
            word_vectors[split[0]] = np.array([float(x) for x in split[1:]])
    return word_vectors


In [54]:
# create embedding matrix
def emb_matrix(model, all_words, word2index):
    matrix_size = len(word_set) + 2
    emb_matrix = np.zeros((matrix_size, 50))
    emb_matrix[0] = np.random.uniform(-0.25, 0.25, 50) # vector for UNK
    emb_matrix[1] = np.zeros(50) # vector for padding, has no weight

    # vector for every other word in the dictionary
    for i in range(2, len(all_words)+2):
        word = all_words[i-2]
        index = word2index[word]
        try:
            vector = model[word]
        except Exception as e:
            # word does not exist in the pretrained embedding
            vector = emb_matrix[0]
        emb_matrix[index] = vector
    return emb_matrix

In [55]:
glv = load_glove_vectors()
data = list(glv.values())
an_array=np.array(data)

In [56]:
embedding = emb_matrix(glv, word_set, word2index)

In [57]:
print(embedding.shape)

(14090, 50)


In [58]:
print(embedding)

[[ 0.21841288 -0.18777747 -0.1930779  ... -0.05635462  0.00554003
  -0.08044043]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.29602    -0.36204    -0.41564    ... -0.24321    -0.62639
  -0.0056615 ]
 ...
 [ 0.044296   -0.23973     0.16029    ... -0.51702    -0.090612
  -0.30921   ]
 [ 0.12015    -1.3559     -0.1136     ...  1.0329     -0.45162
  -0.54243   ]
 [-1.2576     -0.16698     0.80361    ... -0.025248    0.25124
   0.25934   ]]


In [59]:
def validation_metrics(model, valid_dl):
    print("current in vlaidaiton")
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    for x, y, seq_len in valid_dl:
        x = x.long().to(device)
        y = y.long().to(device)
        y_hat = model(x, seq_len)
        loss = criterion(y_hat, y)
        
        # print(f"shape of y: {y.shape}")
        # print(f"shape of y_pred: {y_hat.shape}")
        # print(y_hat)
        pred = torch.argmax(y_hat, 1)
        # print(pred)
        correct += sum((pred == y))
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]

    return sum_loss/total, correct/total

In [60]:
def train_model(model, fname, epochs = 50, lr = 0.0001):
    parameters = filter(lambda p:p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr = lr)
    criterion = nn.CrossEntropyLoss()
    model = model.to(device)
    PATH = '/content/drive/MyDrive/McGill/Comp 550/' + fname
    epoch = 0
    best_val_acc = 0.0
    for i in range(epochs):
        epoch += 1
        print(f"At epoch {i}")
        batch_count = 0
        model.train()
        sum_loss = 0.0
        total = 0.0
        train_correct = 0.0
        for x, y, seq_len in tqdm(trainLoader_glv):
            x = x.long().to(device)
            y = y.long().to(device)
            batch_count += 1
            optimizer.zero_grad()
            y_pred = model(x, seq_len)
            pred = torch.argmax(y_pred, 1)
            train_correct += sum((pred == y))
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = validation_metrics(model, testLoader_glv)
        print("train loss %.3f, train accuracy %.3f, val loss %.3f, val accuracy %.3f" 
              % (sum_loss/total, train_correct/total, val_loss, val_acc))
        if val_acc > best_val_acc and i>=2:
            best_val_acc = val_acc
            NEW_PATH = PATH+f'_{i}_{val_acc}'
            #torch.save(model.state_dict(), NEW_PATH)
            print(f"\t=> Best model saved at {i}th epoch with valication accuracy of {val_acc}")

In [61]:
weight = torch.FloatTensor(an_array)
glv_embedding = nn.Embedding.from_pretrained(weight)

class LSTM_glv(torch.nn.Module):
    def __init__(self, emb_matrix, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(emb_matrix.shape[0], embedding_dim, padding_idx = 0)
        self.embeddings.weight.data.copy_(torch.from_numpy(emb_matrix))
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, seq_len):
        x = self.embeddings(x)
        # x = nn.utils.rnn.pack_padded_sequence(x, self.embedding_dim, batch_first = True)
        x = self.dropout(x)
        # x_padded = pack_padded_sequence(x, seq_len, batch_first = True)
        output, (h_n, c_n) = self.lstm(x)
        # print(h_n[-1].shape)
        # output (N, ave_len, num_class)
        # reshape to out (N, num_class)
        
        return self.linear(h_n[-1])

In [62]:
model = LSTM_glv(embedding, 50, 100)
train_model(model, fname='glv_lstm', epochs=10)

At epoch 0


100%|██████████| 983/983 [00:03<00:00, 251.51it/s]


current in vlaidaiton
train loss 1.226, train accuracy 0.552, val loss 1.130, val accuracy 0.552
At epoch 1


100%|██████████| 983/983 [00:03<00:00, 254.64it/s]


current in vlaidaiton
train loss 1.108, train accuracy 0.553, val loss 1.099, val accuracy 0.552
At epoch 2


...

current in vlaidaiton
train loss 0.952, train accuracy 0.600, val loss 0.900, val accuracy 0.622
	=> Best model saved at 7th epoch with valication accuracy of 0.6224489808082581
At epoch 8


100%|██████████| 983/983 [00:03<00:00, 257.69it/s]


current in vlaidaiton
train loss 0.941, train accuracy 0.608, val loss 0.945, val accuracy 0.602
At epoch 9


100%|██████████| 983/983 [00:03<00:00, 254.49it/s]


current in vlaidaiton
train loss 0.928, train accuracy 0.614, val loss 0.932, val accuracy 0.613
